# Profiling Invasion Percolation

* How long does it take to run?
* Why?

In [45]:
import numpy as np
import random

In [46]:
def percolation(size, spread):
    """
    Simulate invasion percolation on a size x size grid with values in [1..spread],
    reporting density of final filled shape.
    """

    assert (type(size) == int) and ((size > 0) and (size % 2 == 1)), 'size must be positive odd integer'
    assert (type(spread) == int) and (spread > 0), 'spread must be non-negative integer'
    grid = make_grid(size, spread)
    chosen = (int(size/2), int(size/2))
    fill(grid, chosen)
    while not on_boundary(grid, chosen):
        chosen = choose_next(grid)
        fill(grid, chosen)
    return grid, calculate_density(grid)

In [47]:
def make_grid(size, spread):
    """
    Create size x size grid filled with values in [1..spread].
    """
    return np.random.randint(low=1, high=spread+1, size=(size, size))

In [48]:
def fill(grid, loc):
    """
    Mark a cell as filled.
    """
    grid[loc] = 0

In [49]:
def on_boundary(grid, loc):
    """
    Is the specified cell on the boundary of the grid?
    """
    grid_x, grid_y = grid.shape
    loc_x, loc_y = loc
    return (loc_x == 0) or (loc_y == 0) or (loc_x == (grid_x -1)) or (loc_y == (grid_y -1))

def test_on_boundary():
    grid = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    assert on_boundary(grid, (0, 0))
    assert not on_boundary(grid, (1, 1))
    assert on_boundary(grid, (2, 0))

In [50]:
def calculate_density(grid):
    """
    Return proportion of cells that are filled.
    """
    filled = np.sum(grid == 0)
    return filled / grid.size

In [51]:
def choose_next(grid):
    """
    Find and return coordinates of next grid cell to fill.
    """
    candidates = []
    value = 1 + grid.max()
    dim_x, dim_y = grid.shape
    for x in range(dim_x):
        for y in range(dim_y):
            if grid[x, y] == 0:
                pass
            elif is_adjacent(grid, (x, y)):
                if grid[x, y] < value:
                    value = grid[x, y]
                    candidates = [(x, y)]
                elif grid[x, y] == value:
                    candidates.append((x, y))
    return random.choice(candidates)

def test_choose_next():
    assert choose_next(np.array([[9, 1, 9], [9, 0, 9], [9, 9, 9]])) == (0, 1)
    assert choose_next(np.array([[9, 2, 9], [9, 0, 9], [9, 1, 9]])) == (2, 1)

In [52]:
def is_adjacent(grid, loc):
    """
    Is the location (x, y) adjacent to a filled cell?
    """
    x, y = loc
    max_x, max_y = grid.shape
    if grid[loc] == 0:
        return False
    if (x > 0) and (grid[x-1, y] == 0):
        return True
    if (y > 0) and (grid[x, y-1] == 0):
        return True
    if (x < max_x-1) and (grid[x+1, y] == 0):
        return True
    if (y < max_y-1) and (grid[x, y+1] == 0):
        return True
    return False

def test_is_adjacent():
    grid = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
    assert not is_adjacent(grid, (0, 0))
    assert is_adjacent(grid, (1, 0))
    assert is_adjacent(grid, (0, 1))
    assert not is_adjacent(grid, (1, 1))
    assert not is_adjacent(grid, (2, 0))
    assert not is_adjacent(grid, (2, 1))
    assert not is_adjacent(grid, (0, 2))
    assert not is_adjacent(grid, (1, 2))
    assert not is_adjacent(grid, (2, 2))

In [53]:
def test_all():
    test_on_boundary()
    test_is_adjacent()
    test_choose_next()

In [54]:
import timeit

timeit.timeit(stmt='1+2', number=1000)

3.790200571529567e-05

In [55]:
timeit.timeit(stmt='percolation(21, 10)', number=200, setup='from __main__ import percolation')

10.64762847199745

In [56]:
import cProfile
cProfile.run('percolation(141, 10)', sort='tottime')

         16664919 function calls in 59.556 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 16632315   46.030    0.000   46.030    0.000 <ipython-input-52-1981cadaa47b>:1(is_adjacent)
      855   13.453    0.016   59.549    0.070 <ipython-input-51-fce02aa069e4>:1(choose_next)
      856    0.047    0.000    0.047    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    22474    0.006    0.000    0.006    0.000 {method 'append' of 'list' objects}
        1    0.004    0.004   59.556   59.556 <ipython-input-46-e852c9523ac7>:1(percolation)
      855    0.004    0.000    0.010    0.000 random.py:250(choice)
      855    0.004    0.000    0.005    0.000 random.py:220(_randbelow)
      855    0.002    0.000    0.049    0.000 {method 'max' of 'numpy.ndarray' objects}
     1567    0.001    0.000    0.001    0.000 {method 'getrandbits' of '_random.Random' objects}
      856    0.001    0.000    0.001    0.000 <ipython-input-48-68e3576